In [1]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
input_size = 784#here we are going to use PyTorch MNIST dataset. Will talk of custom dataset afterwards. Size of image is 28x28, hence if we make them a 1-D vector then it is 784
output_size = 10#total possible output 0-9
num_epochs = 20
batch_size = 100#total number of images is 60000
learning_rate = 0.001
#here we are using PyTorch dataset hence torchvision.datasets.MNIST, then the image is normal array to convert them to tensor we use transforms.ToTensor()
#later we will see that we can include many other transforms here also.
train_dataset = torchvision.datasets.MNIST(root='../ai61002s19/MNIST', train = True, transform = transforms.ToTensor(), download = True) 
test_dataset = torchvision.datasets.MNIST(root='../ai61002s19/MNIST', train = False, transform= transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)                                          

#in DataLoader we use toch.utils.data.DataLoader(), then we define batch_size and shuffle here. 
#Basically it divides the dataset and loads data in batches

class Logistic(nn.Module):
    def __init__(self, input_size, output_size):
        super(Logistic, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
    def forward(self, x):
        out = self.linear(x)
        return(out)
        
        

model = Logistic(input_size, output_size)

criterion = nn.CrossEntropyLoss()#this could be nn.BCELoss() if we were using it for binary classification
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

print(len(train_dataset))
print(len(train_loader))

total_step = len(train_loader)#this will give total number of iteration.
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):#enumerate is used to give the value of i also. 
        images = images.reshape(-1, 28*28)#600,784
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(epoch%5 == 0):
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Iteration [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) #return max value and the indices
        total += labels.size(0)
        correct += (predicted==labels).sum()
    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 
    

60000
600
Epoch [1/20], Iteration [100/600], Loss: 2.1939
Epoch [1/20], Iteration [200/600], Loss: 2.0933
Epoch [1/20], Iteration [300/600], Loss: 1.9931
Epoch [1/20], Iteration [400/600], Loss: 1.9060
Epoch [1/20], Iteration [500/600], Loss: 1.8546
Epoch [1/20], Iteration [600/600], Loss: 1.8569
Epoch [6/20], Iteration [100/600], Loss: 1.1003
Epoch [6/20], Iteration [200/600], Loss: 0.9031
Epoch [6/20], Iteration [300/600], Loss: 0.9449
Epoch [6/20], Iteration [400/600], Loss: 0.9166
Epoch [6/20], Iteration [500/600], Loss: 0.8977
Epoch [6/20], Iteration [600/600], Loss: 0.8527
Epoch [11/20], Iteration [100/600], Loss: 0.7223
Epoch [11/20], Iteration [200/600], Loss: 0.6693
Epoch [11/20], Iteration [300/600], Loss: 0.7644
Epoch [11/20], Iteration [400/600], Loss: 0.7312
Epoch [11/20], Iteration [500/600], Loss: 0.8660
Epoch [11/20], Iteration [600/600], Loss: 0.6407
Epoch [16/20], Iteration [100/600], Loss: 0.6034
Epoch [16/20], Iteration [200/600], Loss: 0.6197
Epoch [16/20], Iterati

In [4]:
#Write Neural Network By yourself. With HyperParameters

input_size = 784
hidden_size = 100
num_classes = 10
learning_rate = 0.001
num_epochs = 5
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Iteration [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

with torch.no_grad():
    correct = 0 
    total = 0
    for images, labels in (test_loader):
        images= images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total+=predicted.size(0)
        correct += (predicted==labels).sum()
        
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Epoch [1/5], Iteration [100/600], Loss: 0.3462
Epoch [1/5], Iteration [200/600], Loss: 0.3088
Epoch [1/5], Iteration [300/600], Loss: 0.2861
Epoch [1/5], Iteration [400/600], Loss: 0.1597
Epoch [1/5], Iteration [500/600], Loss: 0.3207
Epoch [1/5], Iteration [600/600], Loss: 0.1445
Epoch [2/5], Iteration [100/600], Loss: 0.1394
Epoch [2/5], Iteration [200/600], Loss: 0.1824
Epoch [2/5], Iteration [300/600], Loss: 0.1940
Epoch [2/5], Iteration [400/600], Loss: 0.1417
Epoch [2/5], Iteration [500/600], Loss: 0.0526
Epoch [2/5], Iteration [600/600], Loss: 0.2374
Epoch [3/5], Iteration [100/600], Loss: 0.1548
Epoch [3/5], Iteration [200/600], Loss: 0.1229
Epoch [3/5], Iteration [300/600], Loss: 0.0985
Epoch [3/5], Iteration [400/600], Loss: 0.0805
Epoch [3/5], Iteration [500/600], Loss: 0.1887
Epoch [3/5], Iteration [600/600], Loss: 0.1625
Epoch [4/5], Iteration [100/600], Loss: 0.1755
Epoch [4/5], Iteration [200/600], Loss: 0.0969
Epoch [4/5], Iteration [300/600], Loss: 0.0539
Epoch [4/5], 